In [68]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.model_selection import train_test_split as tts
from sklearn import preprocessing as pp
from sklearn.metrics import f1_score, mean_absolute_error,mean_squared_error , median_absolute_error
from sklearn.metrics import max_error

In [69]:
df = pd.read_csv("iscxIDS2012_subset_no_payload.csv")

In [70]:
df

,Tag,appName,destination,destinationPort,destinationTCPFlagsDescription,direction,protocolName,source,sourceFile,sourcePort,sourceTCPFlagsDescription,startDateTime,stopDateTime,totalDestinationBytes,totalDestinationPackets,totalSourceBytes,totalSourcePackets
0,Attack,Unknown_TCP,131.202.243.84,5555,-,L2R,tcp_ip,192.168.2.112,TestbedMonJun14Flows.xml,3870,S,2010-06-13T23:59:17,2010-06-14T00:00:50,0,0,990,15
1,Attack,Unknown_TCP,131.202.243.84,5555,-,L2R,tcp_ip,192.168.2.112,TestbedMonJun14Flows.xml,3934,S,2010-06-14T00:59:17,2010-06-14T01:00:50,0,0,990,15
2,Attack,Unknown_TCP,131.202.243.84,5555,-,L2R,tcp_ip,192.168.2.112,TestbedMonJun14Flows.xml,3967,S,2010-06-14T01:59:17,2010-06-14T02:00:50,0,0,990,15
3,Attack,Unknown_TCP,131.202.243.84,5555,-,L2R,tcp_ip,192.168.2.112,TestbedMonJun14Flows.xml,4000,S,2010-06-14T02:59:17,2010-06-14T03:00:50,0,0,990,15
4,Attack,SSH,192.168.5.122,22,"S,A",R2L,tcp_ip,218.108.130.167,TestbedMonJun14Flows.xml,51693,"S,R",2010-06-14T03:42:58,2010-06-14T03:42:59,66,1,130,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137815,Normal,HTTPWeb,110.54.110.235,80,"F,S,P,A",L2R,tcp_ip,192.168.1.103,TestbedWedJun16-3Flows.xml,4455,"F,S,P,A",2010-06-16T19:05:03,2010-06-16T19:05:05,5542,7,511,7
137816,Normal,HTTPImageTransfer,72.246.31.18,80,"F,A",L2R,tcp_ip,192.168.4.121,TestbedThuJun17-2Flows.xml,55630,"F,A",2010-06-17T12:11:29,2010-06-17T12:13:15,64,1,128,2
137817,Normal,HTTPImageTransfer,72.11.132.253,80,"F,S,P,A",L2R,tcp_ip,192.168.3.116,TestbedMonJun14Flows.xml,2344,"S,P,A",2010-06-14T15:06:26,2010-06-14T15:06:32,25603,22,1316,18
137818,Normal,DNS,198.164.30.2,53,-,L2R,udp_ip,192.168.5.122,TestbedTueJun15-1Flows.xml,36475,-,2010-06-15T09:20:41,2010-06-15T09:20:41,430,1,92,1


In [71]:
# drop destinationPayloadAsBase64 , destinationPayloadAsUTF, sourcePayloadAsUTF (encrypted) so useless
# Features : appName, destination, destinationPort,destinationTCPFlagsDescription,direction,protocolName,sensorInterfaceId?, sourcePort,sourceTCPFlagsDescription,startDateTime,startTime,stopDateTime,totalDestinationBytes,totalDestinationPackets,totalSourceBytes,totalSourcePackets


In [72]:
for col in df:
  print(df[col].unique())

['Attack' 'Normal']
['Unknown_TCP' 'SSH' 'SecureWeb' 'ICMP' 'HTTPWeb' 'AOL-ICQ' 'Oracle'
 'MiscApplication' 'MSMQ' 'Authentication' 'IRC' 'Filenet' 'Groove'
 'Hotline' 'Real' 'Misc-DB' 'Timbuktu' 'Google' 'OpenNap' 'Yahoo'
 'Misc-Ports' 'ManagementServices' 'Anet' 'XWindows' 'LDAP' 'Flowgen'
 'Squid' 'SNMP-Ports' 'MiscApp' 'NETBEUI' 'Misc-Mail-Port' 'Tacacs' 'MDQS'
 'SMS' 'Hosts2-Ns' 'TimeServer' 'MS-SQL' 'BitTorrent' 'NortonAntiVirus'
 'MSN' 'rexec' 'Network-Config-Ports' 'MicrosoftMediaServer'
 'StreamingAudio' 'Citrix' 'SMTP' 'IPSec' 'NNTPNews' 'Telnet' 'H.323'
 'PostgreSQL' 'rlogin' 'SSL-Shell' 'PCAnywhere' 'Webmin' 'FTP'
 'MSTerminalServices' 'IMAP' 'dsp3270' 'Gnutella' 'Printer' 'Intellex'
 'Ingres' 'rsh' 'POP' 'PPTP' 'WindowsFileSharing' 'SunRPC' 'RPC' 'RTSP'
 'NetBIOS-IP' 'VNC' 'DNS' 'XFER' 'POP-port' 'Common-P2P-Port' 'BGP' 'NFS'
 'HTTPImageTransfer' 'Unknown_UDP' 'WebMediaDocuments' 'WebMediaVideo'
 'NortonGhost' 'WebFileTransfer' 'GuptaSQLBase' 'Web-Port' 'IGMP'
 'Common-Por

In [73]:
from sklearn.preprocessing import OrdinalEncoder

In [74]:
replace_tag = {'Tag': {'Normal': 0, 'Attack': 1}}
replace_tag

{'Tag': {'Normal': 0, 'Attack': 1}}

In [75]:
df = df.replace(replace_tag)
df.Tag.value_counts()

0    68910
1    68910
Name: Tag, dtype: int64

comments
replace_appName = {'appName': {'Unknown_TCP':0, 'SSH':1, 'SecureWeb':2, 'ICMP':3, 'HTTPWeb':4, 'AOL-ICQ':5, 'Oracle':6, 
 'MiscApplication':7, 'MSMQ':8, 'Authentication':9, 'IRC':10, 'Filenet':11, 'Groove':12,
 'Hotline':13, 'Real':14, 'Misc-DB':15, 'Timbuktu':16, 'Google':17, 'OpenNap':18, 'Yahoo':19,
 'Misc-Ports':20, 'ManagementServices':21, 'Anet':22, 'XWindows':23, 'LDAP':24, 'Flowgen':25,
 'Squid':26, 'SNMP-Ports':27, 'MiscApp':28, 'NETBEUI':29, 'Misc-Mail-Port':30, 'Tacacs':31, 'MDQS':32,
 'SMS':33, 'Hosts2-Ns':34, 'TimeServer':35, 'MS-SQL':36, 'BitTorrent':37, 'NortonAntiVirus':38,
 'MSN':39, 'rexec':40, 'Network-Config-Ports':41, 'MicrosoftMediaServer':42,
 'StreamingAudio':43, 'Citrix':44, 'SMTP':45, 'IPSec':46, 'NNTPNews':47, 'Telnet':48, 'H.323':49,
 'PostgreSQL':50, 'rlogin':51, 'SSL-Shell':52, 'PCAnywhere':53, 'Webmin':54, 'FTP':55,
 'MSTerminalServices':56, 'IMAP':57, 'dsp3270':58, 'Gnutella':59, 'Printer':60, 'Intellex':61,
 'Ingres':62, 'rsh':63, 'POP':64, 'PPTP':65, 'WindowsFileSharing':66, 'SunRPC' :67,'RPC':68, 'RTSP':69,
 'NetBIOS-IP':70, 'VNC':71, 'DNS':72, 'XFER':73, 'POP-port':74, 'Common-P2P-Port':75, 'BGP':76, 'NFS':77,
 'HTTPImageTransfer':78, 'Unknown_UDP':79, 'WebMediaDocuments':80, 'WebMediaVideo':81,
 'NortonGhost':82, 'WebFileTransfer':83, 'GuptaSQLBase':84, 'Web-Port':85, 'IGMP':86,
 'Common-Ports':87, 'DNS-Port':87, 'giop-ssl':88, 'iChat':89, 'WebMediaAudio':90, 'MSN-Zone':91,
 'SSDP':92, 'OpenWindows':93}}

In [76]:

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
transformed_data = df.copy()

for col in transformed_data.columns:
    if transformed_data[col].dtype == 'object':
        transformed_data[col] = le.fit_transform(transformed_data[col].values)
transformed_data.head(10)

,Tag,appName,destination,destinationPort,destinationTCPFlagsDescription,direction,protocolName,source,sourceFile,sourcePort,sourceTCPFlagsDescription,startDateTime,stopDateTime,totalDestinationBytes,totalDestinationPackets,totalSourceBytes,totalSourcePackets
0,1,76,554,5555,0,1,3,71,0,3870,17,13358,13617,0,0,990,15
1,1,76,554,5555,0,1,3,71,0,3934,17,13484,13734,0,0,990,15
2,1,76,554,5555,0,1,3,71,0,3967,17,13655,13911,0,0,990,15
3,1,76,554,5555,0,1,3,71,0,4000,17,13903,14161,0,0,990,15
4,1,66,1411,22,17,2,3,137,0,51693,20,14046,14299,66,1,130,2
5,1,66,1411,22,7,2,3,137,0,54053,11,14069,14325,2651,14,1304,12
6,1,76,554,5555,0,1,3,71,0,4039,17,14075,14333,0,0,990,15
7,1,76,554,5555,0,1,3,71,0,4558,17,14333,14601,0,0,990,15
8,1,76,554,5555,0,1,3,71,0,1562,17,14720,14983,0,0,990,15
9,1,76,554,5555,0,1,3,71,0,2030,17,15069,15327,0,0,990,15


In [77]:
transformed_data.appName.value_counts()

19    64454
18    24796
76    14989
8     10930
66     7498
      ...  
49        1
90        1
51        1
81        1
16        1
Name: appName, Length: 95, dtype: int64

In [78]:
x = transformed_data.drop(['Tag'], axis=1).copy()
y = transformed_data["Tag"]

In [79]:
 #x_train,x_test,y_train,y_test = tts(x,y, test_size=.4,random_state=0)

In [80]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

In [81]:
clf = DecisionTreeClassifier(random_state=0)
cross_val_score(clf, x, y, cv=10)

array([0.67392251, 0.98650414, 0.99963721, 0.99970977, 0.99934697,
       0.99876651, 0.98984182, 0.67174576, 0.99985488, 0.94942679])

In [82]:
apply(X, check_input=True)

NameError: name 'apply' is not defined